In [2]:
!pip install gensim

In [3]:
import gensim.downloader as api
from gensim.models import KeyedVectors

In [59]:
#to avoid it downloading agai
#wv = api.load('word2vec-google-news-300')

In [4]:
#lets save the wv in google drive to get when reconnecting instead of downloading again

saved_path = '/content/drive/MyDrive/NLP_downloaded_models/word2vec-google-news-300.model'

#wv.save(saved_path)

#to load this model
wv = KeyedVectors.load(saved_path)

In [5]:
wv.similarity('queen','king')

0.6510957

In [6]:
wv.most_similar('engineering')

[('Engineering', 0.7129400372505188),
 ('mechanical_engineering', 0.6925394535064697),
 ('engineer', 0.6766179203987122),
 ('electrical_engineering', 0.6710788011550903),
 ('engineers', 0.6125999689102173),
 ('aeronautical_engineering', 0.5867857336997986),
 ('Invensys_Plc_ISYS_LN', 0.5768389701843262),
 ('geotechnical_engineering', 0.5687716603279114),
 ('engi_neering', 0.5667977929115295),
 ('design', 0.564548671245575)]

In [7]:
wv.vector_size

300

In [8]:
queen = wv['queen']
king = wv['king']
man = wv['man']
woman = wv['woman']

queen_calculated = king-man+woman

In [9]:
man.shape , queen_calculated.shape

((300,), (300,))

In [10]:
def reshaped_vector(vector):
  return vector.reshape(1,-1)

In [11]:
queen_calculated = reshaped_vector(queen_calculated)
queen = reshaped_vector(queen)

In [12]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(queen_calculated, queen)

array([[0.7300518]], dtype=float32)

In [13]:
wv.most_similar(positive=['india','colombo'], negative=['delhi'], topn=5)

[('sri_lanka', 0.5586314797401428),
 ('tamils', 0.5541576743125916),
 ('sri_lankan', 0.5413911938667297),
 ('ltte', 0.5336052775382996),
 ('srilanka', 0.5257565975189209)]

In [14]:
text1 = 'Hello guys lets go to visit in Sri Lanka this weekend'
text2 = 'Hello shall we go to travel this weekend in Colombo'

In [15]:
def text_to_vector (text) :
  vectors = []
  for word in text:
    if word in wv.key_to_index:
      vectors.append(wv[word])

  text_vec =  sum(vectors) / len(vectors)
  return reshaped_vector(text_vec)

In [16]:
text_1_vec = text_to_vector(text1)
text_2_vec = text_to_vector(text2)

text_1_vec.shape , text_2_vec.shape

((1, 300), (1, 300))

In [17]:
cosine_similarity(text_1_vec, text_2_vec)

array([[0.9805964]], dtype=float32)

In [25]:
#lets create a function to get the similerty by just inputing 2 text

def similarlity_of_2text (text1, text2):
  text_1_vec = text_to_vector(text1)
  text_2_vec = text_to_vector(text2)
  similarlity = cosine_similarity(text_1_vec, text_2_vec)
  print(f'Similarlity : {similarlity[0][0]*100:.2f} %')

In [26]:
similarlity_of_2text('Deshani did you get that point during the lecture ?',
                     'Deshani understand the lecture very well during the lecture')

Similarlity : 98.11 %


In [27]:
similarlity_of_2text('Yesterday We wend to the playground with some of police oficers and they saw us?',
                     '12, july 2024 we went to the ground and when we going we bring some police offiers with us and the thief saw us comming')

Similarlity : 98.80 %


In [28]:
#we can try glove models too but the weight ( size) of them are bit higher and have to save data

In [36]:
import numpy as np
import pandas as pd
import spacy

In [31]:
df = pd.read_csv('/content/drive/MyDrive/Dataset/fake_and_real_news.csv')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9900 entries, 0 to 9899
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    9900 non-null   object
 1   label   9900 non-null   object
dtypes: object(2)
memory usage: 154.8+ KB


In [33]:
df.head()

,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [34]:
df['label_num'] = df['label'].map({'Fake': 0, 'Real':1})

In [35]:
df['label_num'].value_counts()

,count
label_num,
0,5000
1,4900


In [77]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 778.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [80]:
nlp = spacy.load('en_core_web_lg')

def preprocessing (text):
  doc = nlp(text)

  filtered_tokens = []

  for token in doc:
    if not token.is_stop and not token.is_punct:
      filtered_tokens.append(token.lemma_.lower())

  return wv.get_mean_vector(filtered_tokens)

In [81]:
df['vector'] = df['Text'].apply(preprocessing)

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

X_train , X_test, y_train, y_test = train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2,
    random_state=2022,
    stratify= df.label_num
)

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [84]:
scaler = MinMaxScaler()
X_train_2d_scaled = scaler.fit_transform(X_train_2d)
X_test_2d_scaled = scaler.transform(X_test_2d)

In [85]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_2d_scaled, y_train)

MultinomialNB()

In [86]:
from sklearn.metrics import classification_report

y_pred = clf.predict(X_test_2d_scaled)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.91      0.91      1000
           1       0.91      0.92      0.91       980

    accuracy                           0.91      1980
   macro avg       0.91      0.91      0.91      1980
weighted avg       0.91      0.91      0.91      1980



In [87]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier()
clf.fit(X_train_2d, y_train)

y_pred = clf.predict(X_test_2d)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.97      0.98      1000
           1       0.97      0.98      0.98       980

    accuracy                           0.98      1980
   macro avg       0.98      0.98      0.98      1980
weighted avg       0.98      0.98      0.98      1980



In [82]:
saved_path_for_lg_english = '/content/drive/MyDrive/NLP_downloaded_models/en_core_web_lg_saved'
nlp.to_disk(saved_path_for_lg_english)

Seems Like its grate with the accuracy of 98%